In [ ]:
pip install langgraph==0.3.5

https://langchain-ai.github.io/langgraphjs/reference/classes/langgraph.StateGraph.html

In [ ]:
from langgraph.graph import StateGraph

In [ ]:
def merge_dicts(dict1, dict2):
    return {**dict1, **dict2}

In [ ]:
from typing_extensions import TypedDict
import operator
from typing import Annotated

class State(TypedDict):
    text: str
    answer: Annotated[dict, merge_dicts]
    payload: dict[str, list]
    tag: str 

In [ ]:
graph_builder = StateGraph(State)


In [ ]:
def extract_content(state: State):
    return {"text": state["payload"][0]["customer_remark"]}

def route_question_or_compliment(state: State):
    if "?" in state["text"]:
        return "question"
    else:
        return "compliment"


def run_compliment_code(state: State):
    return {"answer": {"temp_answer": "Thanks for the compliment."}}


def run_question_code(state: State):
    return {"answer": {"temp_answer": "Wow nice question."}}


def beautify(state: State):
    return {
        "answer": {"final_beautified_answer": 
          [
           str(state["answer"]["temp_answer"])
         + f'I will pass it to the {state["tag"]} Department' ## newly added
            ]
        }
    }

def tag_query(state: State):
    if "package" in state["text"]:
        return {"tag": "Packaging"}
    elif "price" in state["text"]:
        return {"tag": "Pricing"}
    else:
        return {"tag": "General"}

In [ ]:
graph_builder.add_node("extract_content", extract_content)
graph_builder.add_node("run_question_code", run_question_code)
graph_builder.add_node("run_compliment_code", run_compliment_code)
graph_builder.add_node("beautify", beautify)
graph_builder.add_node("tag_query", tag_query)

In [ ]:
from langgraph.graph import END, START

In [ ]:
graph_builder.add_edge(START, "extract_content")
graph_builder.add_conditional_edges(
    "extract_content",
    route_question_or_compliment,
    {
        "compliment": "run_compliment_code",
        "question": "run_question_code",
    },
)
graph_builder.add_edge("run_question_code", "beautify")
graph_builder.add_edge("run_compliment_code", "beautify")
graph_builder.add_edge("beautify", END)
graph_builder.add_edge("tag_query", "beautify")



In [ ]:
graph = graph_builder.compile()

In [ ]:
graph

In [ ]:
graph.invoke({
        "payload": [
            {
                "time_of_comment": "20-01-2025",
                "customer_remark": "I hate this.",
                "social_media_channel": "facebook",
                "number_of_likes": 100,
            }
        ]
    })

If you need the step-by-step response at each node, you can use stream(). This is useful if you want to send updates to the front end to show a progress bar.

In [ ]:
for step in graph.stream({"payload": [{"time_of_comment": "20-01-2025",
                            "customer_remark": "I hate this.", 
                             "social_media_channel": "facebook",
                              "number_of_likes":100
                            }]
}):
    print(step)